## Work
1. 請嘗試將 preproc_x 替換成以每筆資料的 min/max 進行標準化至 -1 ~ 1 間，再進行訓練
2. 請嘗試將 mlp 疊更深 (e.g 5~10 層)，進行訓練後觀察 learning curve 的走勢
3. (optional) 請改用 GPU 進行訓練 (如果你有 GPU 的話)，比較使用 CPU 與 GPU 的訓練速度

In [1]:
## 
"""
Your code here (optional)
確認硬體資源
"""


In [9]:
import os
import keras
from sklearn.preprocessing import MinMaxScaler

# 請嘗試設定 GPU：os.environ


In [10]:
train, test = keras.datasets.cifar10.load_data()

In [11]:
## 資料前處理
"""
Your code here
"""
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    x = MinMaxScaler(feature_range=(-1, 1)).fit_transform(x)
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [12]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [15]:
input_shape = x_train.shape[1:]
output_units = 10
hidden_unit_dict = {1:512,2:256,3:128}

In [17]:
"""Code Here
建立你的神經網路
"""
def build_mlp(input_shape, output_units, hidden_unit_dict):
    # input layer
    input_layer = keras.layers.Input(input_shape)
    
    # hidden layer
    for hidden_layer in hidden_unit_dict:
        if hidden_layer == 1:
            x = keras.layers.Dense(units=hidden_unit_dict[hidden_layer], 
                                   activation="relu", 
                                   name="hidden_layer"+str(hidden_layer)
                                  )(input_layer)
        else:
            x = keras.layers.Dense(units=hidden_unit_dict[hidden_layer], 
                                   activation="relu", 
                                   name="hidden_layer"+str(hidden_layer)
                                  )(x)
            
    # output layer       
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

# build model    
model = build_mlp(input_shape, output_units, hidden_unit_dict)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________


In [18]:
## 超參數設定
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 256

In [19]:
optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

In [ ]:
model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 12s 245us/step - loss: 1.6612 - accuracy: 0.4128 - val_loss: 1.5239 - val_accuracy: 0.4595
Epoch 2/100
50000/50000 [==============================] - 11s 220us/step - loss: 1.4354 - accuracy: 0.4912 - val_loss: 1.4639 - val_accuracy: 0.4796
Epoch 3/100
50000/50000 [==============================] - 11s 219us/step - loss: 1.3194 - accuracy: 0.5362 - val_loss: 1.4033 - val_accuracy: 0.5102
Epoch 4/100
50000/50000 [==============================] - 11s 216us/step - loss: 1.2257 - accuracy: 0.5693 - val_loss: 1.3705 - val_accuracy: 0.5195
Epoch 5/100
50000/50000 [==============================] - 11s 224us/step - loss: 1.1388 - accuracy: 0.6004 - val_loss: 1.3826 - val_accuracy: 0.5213
Epoch 6/100
50000/50000 [==============================] - 11s 220us/step - loss: 1.0517 - accuracy: 0.6289 - val_loss: 1.3828 - val_accuracy: 0.5292
Epoch 7/100
50000/50000 [=========================

In [ ]:
import matplotlib.pyplot as plt


train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]

train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()